## Some Calibration Theory

My aim is to calibrate the Merton jump-diffusion (MJD) model, an extension of the Black-Scholes model to account for jumps in stock prices. It combines continuous diffusion and discrete jumps to model the price dynamics of a risky asset. 

The SDE of MJD model:
$$\frac{dS_t}{S_t} =  r dt+\sigma dW_t+(e^{\nu+\delta z}-1) dN_t,$$
where $S_t$ is stock price at time $t$, $r$ is the risk-free rate, $\sigma$ is the volatility of the stock, $W_t$ is a standard Browian motion which drives the diffusion part of the process, $dN_t$ is a Poisson process with intensity $\lambda$ which represents the number of jumps, $e^{\nu+\delta z}$ is the jump size with $\nu$ being the mean jump size, $\delta$ is the standard deviation of the jump size, and $z\sim \mathcal{N} (0,1)$ is a standard normal random variable.


Calibration will be done on an artificial implied volatility surface of 25 option contracts. The objective function is given by:
$$
\mathcal{L}(\lambda, \nu, \delta) = \sum_{i=1}^{N_T} \sum_{j=1}^{N_X} w_{i,j}\left( C_{\text{MJD}}(S_0, X_j, T_i, r, \sigma, \lambda, \nu, \delta) - C_{\text{observed}}(X_j, T_i) \right)^2,
$$
where observed call price is obtained via Black-Scholes-Merton:
$$
C_{\text{BSM}}(S_0, X, T, r, \sigma) = S_0 N(d_1) - X e^{-rT} N(d_2),
$$

$$
d_1 = \frac{\ln\left(\frac{S_0}{X}\right) + \left(r + \frac{1}{2} \sigma^2\right) T}{\sigma \sqrt{T}},
$$

$$
d_2 = d_1 - \sigma \sqrt{T},
$$
and MJD model uses:
$$
C_{\text{MJD}}(S_0, X, T, r, \sigma, \lambda, \nu, \delta) = \sum_{k=0}^{\infty} \frac{e^{-\lambda T} (\lambda T)^k}{k!} C_{\text{BSM}}\left( S_0, X, T, r_k, \sigma_k \right),
$$

$$
\sigma_k = \sqrt{\sigma^2 + k \delta^2},
$$

$$
r_k = r - \lambda \left( e^{\nu + \frac{1}{2} \delta^2} - 1 \right).
$$
There are various ways of computing weights $w_{i,j}$, one can even just set it up to one. In this tutorial, I use:
$$w_{i,j} = \frac{1}{C_{\text{observed}}(X_j, T_i)}.$$

## Volatility Surface Used

In [1]:
import numpy as np

S0 = 11
r = 0.06

strike_prices = np.array([10, 10.5, 11, 11.5, 12])
time_to_maturity = np.array([0.1, 0.2, 0.3, 0.4, 0.5])

imp_vol_surface = np.array([
    [0.349317, 0.322159, 0.296554, 0.272501, 0.250000],  # T = 0.1
    [0.349488, 0.322338, 0.296740, 0.272695, 0.250202],  # T = 0.2
    [0.349658, 0.322516, 0.296926, 0.272889, 0.250404],  # T = 0.3
    [0.349829, 0.322695, 0.297113, 0.273083, 0.250605],  # T = 0.4
    [0.350000, 0.322873, 0.297299, 0.273277, 0.250807],  # T = 0.5
])

## Calibration Steps

In [2]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from math import exp, factorial
import pandas as pd

In [3]:
# Step 1: Black-Scholes formula to calculate call prices
def black_scholes_call(S0, X, T, r, sigma):
    d1 = (np.log(S0 / X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d1 - sigma * np.sqrt(T))

# Calculate Black-Scholes prices using the implied volatility surface
bs_prices = np.zeros_like(imp_vol_surface)
for i, T in enumerate(time_to_maturity):
    for j, X in enumerate(strike_prices):
        sigma = imp_vol_surface[i, j]  # Implied volatility
        bs_prices[i, j] = black_scholes_call(S0, X, T, r, sigma)

# Step 2: Merton Jump Diffusion model
def merton_jump_call(S0, X, T, r, sigma, lambda_, nu, delta, max_jumps=50):
    return sum(
        exp(-lambda_ * T) * (lambda_ * T)**k / factorial(k) * 
        black_scholes_call(S0, X, T, r - lambda_ * (exp(nu + 0.5 * delta**2) - 1), np.sqrt(sigma**2 + k * delta**2))
        for k in range(max_jumps)
    )

# Objective function: minimize the difference between Merton prices and Black-Scholes prices
def objective(params):
    lambda_, nu, delta = params
    return sum(
        (1 / bs_prices[i, j] if bs_prices[i, j] != 0 else 1) * 
        (merton_jump_call(S0, X, T, r, imp_vol_surface[i, j], lambda_, nu, delta) - bs_prices[i, j])**2
        for i, T in enumerate(time_to_maturity)
        for j, X in enumerate(strike_prices)
    )

# Initial guesses for lambda (jump intensity), nu (mean jump size), and delta (jump size volatility)
initial_params = [0.1, 0.0, 0.2]

# Perform the optimization using L-BFGS-B with price inverse weights
result = minimize(objective, initial_params, bounds=[(0, 1), (-1, 1), (0, 1)], method='L-BFGS-B')

# Extract the calibrated parameters
lambda_calibrated, nu_calibrated, delta_calibrated = result.x

print(f"Calibrated Lambda (Jump Intensity): {np.round(lambda_calibrated, 4)}")
print(f"Calibrated Nu (Mean Jump Size): {np.round(nu_calibrated, 4)}")
print(f"Calibrated Delta (Jump Size Volatility): {np.round(delta_calibrated, 4)}")

Calibrated Lambda (Jump Intensity): 0.0004
Calibrated Nu (Mean Jump Size): 0.0054
Calibrated Delta (Jump Size Volatility): 0.1518


In [4]:
print("Black-Scholes Prices:")
print(np.round(bs_prices, 4))

Black-Scholes Prices:
[[1.1721 0.7734 0.4439 0.2076 0.0719]
 [1.357  0.9765 0.6464 0.3814 0.1917]
 [1.5179 1.143  0.809  0.527  0.307 ]
 [1.6621 1.289  0.9507 0.6565 0.4155]
 [1.7944 1.4214 1.079  0.7751 0.5182]]


In [5]:
# Calculate Merton prices using calibrated parameters
merton_prices = np.zeros_like(bs_prices)

for i, T in enumerate(time_to_maturity):
    for j, X in enumerate(strike_prices):
        merton_prices[i, j] = merton_jump_call(S0, X, T, r, imp_vol_surface[i, j], lambda_calibrated, nu_calibrated, delta_calibrated)

print("Merton Jump Diffusion Prices (after calibration):")
print(np.round(merton_prices, 4))

Merton Jump Diffusion Prices (after calibration):
[[1.1721 0.7734 0.4439 0.2076 0.0719]
 [1.357  0.9765 0.6464 0.3814 0.1918]
 [1.5179 1.143  0.809  0.527  0.307 ]
 [1.6621 1.289  0.9507 0.6565 0.4155]
 [1.7944 1.4214 1.079  0.7752 0.5182]]


In [6]:
# Calculate the difference between Merton and Black-Scholes prices
price_difference = merton_prices - bs_prices

print("Difference between Merton and Black-Scholes Prices:")
print(price_difference)

Difference between Merton and Black-Scholes Prices:
[[-4.24164410e-06 -3.14140449e-06 -1.72725976e-06 -3.87535876e-07
   3.87506672e-07]
 [-5.92978382e-06 -4.05664468e-06 -1.85361485e-06  3.01528830e-07
   1.86654341e-06]
 [-6.45904563e-06 -3.90085068e-06 -9.58724026e-07  1.99188432e-06
   4.37405081e-06]
 [-6.16390227e-06 -2.93926374e-06  7.42968054e-07  4.51011219e-06
   7.76177454e-06]
 [-5.20773118e-06 -1.31394107e-06  3.12400208e-06  7.73841109e-06
   1.19142462e-05]]


## Calibration Steps

From this price difference you can see that we are able to price back the MJD to the artifical market prices. In real world, we need to use real data rather than artificial one. After calibrating we can then use these parameters to price any derivative or option that follows the MJD SDE using simulation.